# Description

It combines all clustering solutions generated into a single consolidated solution using consensus clustering.

# Environment variables

In [1]:
from IPython.display import display

# set numpy n_jobs to 1, since I'll be using n_jobs later
NUMPY_N_JOBS = 1
display(NUMPY_N_JOBS)

1

In [2]:
%env MKL_NUM_THREADS=$NUMPY_N_JOBS
%env OPEN_BLAS_NUM_THREADS=$NUMPY_N_JOBS
%env NUMEXPR_NUM_THREADS=$NUMPY_N_JOBS
%env OMP_NUM_THREADS=$NUMPY_N_JOBS

env: MKL_NUM_THREADS=1
env: OPEN_BLAS_NUM_THREADS=1
env: NUMEXPR_NUM_THREADS=1
env: OMP_NUM_THREADS=1


# Modules loading

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from pathlib import Path

import numpy as np
import pandas as pd

import conf

# Settings

In [5]:
RANDOM_STATES_ITER = iter(range(0, 10000000))

## Consensus clustering

In [6]:
CLUSTERING_OPTIONS = {}

CLUSTERING_OPTIONS["K_MIN"] = 2
CLUSTERING_OPTIONS["K_MAX"] = 60

display(CLUSTERING_OPTIONS)

{'K_MIN': 2, 'K_MAX': 60}

In [7]:
# output dir for this notebook
RESULTS_DIR = Path(conf.RESULTS["CLUSTERING_DIR"], "consensus_clustering").resolve()
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

display(RESULTS_DIR)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering')

# Load ensemble

In [8]:
output_file = Path(RESULTS_DIR, "ensemble.npy").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering/ensemble.npy')

In [9]:
full_ensemble = np.load(output_file)

In [10]:
display(full_ensemble.shape)

(4428, 3749)

# Load ensemble coassociation distance matrix

In [11]:
output_file = Path(RESULTS_DIR, "ensemble_coassoc_matrix.npy").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering/ensemble_coassoc_matrix.npy')

In [12]:
ensemble_coassoc_matrix = np.load(output_file)

In [13]:
display(ensemble_coassoc_matrix.shape)

(3749, 3749)

In [14]:
display(ensemble_coassoc_matrix)

array([[0.        , 0.05615942, 0.05389493, ..., 0.59986413, 0.66171429,
        0.66757246],
       [0.05615942, 0.        , 0.02740036, ..., 0.60122283, 0.65988571,
        0.66530797],
       [0.05389493, 0.02740036, 0.        , ..., 0.59782609, 0.66102857,
        0.66281703],
       ...,
       [0.59986413, 0.60122283, 0.59782609, ..., 0.        , 0.6699339 ,
        0.66531165],
       [0.66171429, 0.65988571, 0.66102857, ..., 0.6699339 , 0.        ,
        0.58650558],
       [0.66757246, 0.66530797, 0.66281703, ..., 0.66531165, 0.58650558,
        0.        ]])

# Consensus clustering

In [15]:
from concurrent.futures import ProcessPoolExecutor, as_completed

from tqdm import tqdm

from clustering.ensembles.utils import (
    run_method_and_compute_agreement,
)
from clustering.ensembles.eac import (
    eac_single_coassoc_matrix,
    eac_complete_coassoc_matrix,
    eac_average_coassoc_matrix,
)
from clustering.ensembles.spectral import scc

Define spectral consensus clustering methods with delta values found in pre-analysis:

In [16]:
def scc_020(coassoc_distance_matrix, k, **kwargs):
    return scc(
        coassoc_distance_matrix,
        k,
        delta=0.20,
        ensemble_is_coassoc_matrix=True,
        **kwargs
    )


def scc_025(coassoc_distance_matrix, k, **kwargs):
    return scc(
        coassoc_distance_matrix,
        k,
        delta=0.25,
        ensemble_is_coassoc_matrix=True,
        **kwargs
    )


def scc_030(coassoc_distance_matrix, k, **kwargs):
    return scc(
        coassoc_distance_matrix,
        k,
        delta=0.30,
        ensemble_is_coassoc_matrix=True,
        **kwargs
    )


def scc_050(coassoc_distance_matrix, k, **kwargs):
    return scc(
        coassoc_distance_matrix,
        k,
        delta=0.50,
        ensemble_is_coassoc_matrix=True,
        **kwargs
    )

In [17]:
all_consensus_methods = set(
    (
        eac_single_coassoc_matrix,
        eac_complete_coassoc_matrix,
        eac_average_coassoc_matrix,
        scc_020,
        scc_025,
        scc_030,
        scc_050,
    )
)
display(all_consensus_methods)

{<function clustering.ensembles.eac.eac_average_coassoc_matrix(coassoc_matrix, k, **kwargs)>,
 <function clustering.ensembles.eac.eac_complete_coassoc_matrix(coassoc_matrix, k, **kwargs)>,
 <function clustering.ensembles.eac.eac_single_coassoc_matrix(coassoc_matrix, k, **kwargs)>,
 <function __main__.scc_020(coassoc_distance_matrix, k, **kwargs)>,
 <function __main__.scc_025(coassoc_distance_matrix, k, **kwargs)>,
 <function __main__.scc_030(coassoc_distance_matrix, k, **kwargs)>,
 <function __main__.scc_050(coassoc_distance_matrix, k, **kwargs)>}

In [18]:
consensus_results = []

with ProcessPoolExecutor(max_workers=conf.GENERAL["N_JOBS"]) as executor:
    tasks = {
        executor.submit(
            run_method_and_compute_agreement,
            m,
            ensemble_coassoc_matrix,
            full_ensemble,
            k,
            random_state=next(RANDOM_STATES_ITER),
        ): (m.__name__, k)
        for m in all_consensus_methods
        for k in range(CLUSTERING_OPTIONS["K_MIN"], CLUSTERING_OPTIONS["K_MAX"] + 1)
    }

    for future in tqdm(as_completed(tasks), total=len(tasks), disable=False, ncols=100):
        method_name, k = tasks[future]
        part, performance_values = future.result()

        method_results = {
            "method": method_name,
            "partition": part,
            "k": k,
        }
        method_results.update(performance_values)

        consensus_results.append(method_results)

  0%|                                                                       | 0/413 [00:00<?, ?it/s]

  0%|▏                                                            | 1/413 [00:23<2:39:31, 23.23s/it]

  0%|▎                                                            | 2/413 [00:24<1:53:18, 16.54s/it]

  1%|▍                                                            | 3/413 [00:48<2:08:05, 18.74s/it]

  1%|▌                                                            | 4/413 [00:49<1:31:35, 13.44s/it]

...

 99%|██████████████████████████████████████████████████████████▌| 410/413 [2:22:52<01:07, 22.34s/it]

100%|██████████████████████████████████████████████████████████▋| 411/413 [2:23:01<00:36, 18.48s/it]

100%|██████████████████████████████████████████████████████████▊| 412/413 [2:23:34<00:22, 22.60s/it]

100%|███████████████████████████████████████████████████████████| 413/413 [2:23:43<00:00, 18.68s/it]

100%|███████████████████████████████████████████████████████████| 413/413 [2:23:43<00:00, 20.88s/it]

In [19]:
consensus_results = pd.DataFrame(consensus_results)

In [20]:
display(consensus_results.shape)

(413, 12)

In [21]:
consensus_results.head()

,method,partition,k,ari_mean,ari_median,ari_std,ami_mean,ami_median,ami_std,nmi_mean,nmi_median,nmi_std
0,eac_average_coassoc_matrix,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2,0.226733,0.010542,0.351673,0.238194,0.055225,0.325239,0.240435,0.057239,0.324510
1,eac_average_coassoc_matrix,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3,0.242829,0.015836,0.359734,0.266882,0.080544,0.325173,0.270462,0.083603,0.323833
2,eac_average_coassoc_matrix,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",4,0.260508,0.032736,0.339536,0.296430,0.131213,0.279790,0.301305,0.135507,0.277920
3,eac_average_coassoc_matrix,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5,0.260519,0.032744,0.339554,0.298287,0.132804,0.281202,0.303842,0.138038,0.278911
4,eac_average_coassoc_matrix,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",6,0.165668,0.091730,0.200208,0.268626,0.236109,0.128800,0.275338,0.237406,0.128180


## Testing

In [22]:
assert not consensus_results.isna().any().any()

In [23]:
# check that the number of clusters in the partitions are the expected ones
_real_k_values = consensus_results["partition"].apply(lambda x: np.unique(x).shape[0])
display(_real_k_values)
assert np.all(consensus_results["k"].values == _real_k_values.values)

0       2
1       3
2       4
3       5
4       6
       ..
408    56
409    57
410    58
411    59
412    60
Name: partition, Length: 413, dtype: int64

## Save

In [24]:
output_file = Path(RESULTS_DIR, "consensus_clustering_runs.pkl").resolve()
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base3/results/clustering/consensus_clustering/consensus_clustering_runs.pkl')

In [25]:
consensus_results.to_pickle(output_file)